# Build Your Own Search Engine

The success of Google can be boiled down to this simple equation:

    "information retrieval" + "linear algebra / graph analytics" + "machine learning" = $$$

In a little more detail, these key ingredients are:

- indexing and keyword search for organizing enormous collection of web documents
- the PageRank algorithm for computing importance of web pages (for ranking search results)
- learning to rerank results based on user clicks (and other features)

We're going to work on part 1 of the equation by developing our own search engine for indexing and retrieving documents via boolean search queryies. That just leads to one dollar sign:

    "information retrieval" = $

But it should still be fun!

## 0 Imports and Helpers

In [7]:
from collections import defaultdict, Counter
import nltk
from nltk.tokenize import word_tokenize
import numpy as np
import string
import time
from operator import itemgetter



AttributeError: module 'nltk' has no attribute 'data'

In [8]:
# run NLTK downloader to get Punkt sentence chunking model
# look for "punkt" under "Models" tab in NLTK Downloader gui that pops up
nltk.download()

NameError: name 'nltk' is not defined

## 1 MySearchEngine class

We're going to be creating a single class that maintains an in-memory index for adding, removing, and querying for documents. We'll assume that documents have an associated id that can be used to uniquely identify it (e.g., a URL for a web document). For this search engine, we'll be using the following definitions:

- collection of documents: $D$
- number of documents in collection: $N = \lvert D \rvert$
- term frequency (number of times term $t$ appears in document $d$): $tf(t, d)$
- document frequency of a term (number if documents that contain term $t$): $df(t, D) = \lvert \{d \in D : t \in d\} \rvert$
- inverse document frequency: $idf(t, D) = \log_{10} \left( \frac{N}{1 + df(t, D)} \right)$
- term frequency-inverse document frequency: $tfidf(t, d, D) = tf(t, d) \cdot idf(t, D)$
- dot product between two documents: $d_1 \cdot d_2 = \sum_{t} tfidf(t, d_1, D) \cdot tfidf(t, d_2, D)$
- length of document: $\lVert d \rVert = \sqrt{\sum_{t} tfidf(t, d, D)}$
- cosine similarity between two documents: $sim(d_1, d_2) = \frac{d_1 \cdot d_2}{\lVert d_1 \rVert \cdot \lVert d_2 \rVert}$

Note: This search engine will be dynamic in the sense that documents can be added and removed at any time. One consequence is that we won't be precomputing $tfidf(t, d)$ for terms in a document $d$ because $idf(t, D)$ will change as the document collection changes. Instead, we'll just be maintaining $tf(t, d)$ values for documents, and multiplying by the current value of $idf(t, D)$ as needed (according to definitions above).

A second consequence is that we won't be storing the term frequencies for a document as a fixed length vector, again, because the vocabulary will be changing as documents get added or removed. Instead, we'll be storing term frequencies as Counter objects (dictionaries). This is like using a sparse vector to represent the term frequencies (but in a position-independent way). Now computing the dot product will involve finding which terms occur in both documents, since only those will affect the dot product. (If a term is missing from one of the documents, the product of the two counts will be zero.)

Tip: Try completing this notebook in the style of "test-driven development" (see https://en.wikipedia.org/wiki/Test-driven_development). That is, just implement enough of the methods in the class to get the initial tests to pass, then implement some more to get the next set of tests to pass, and so on. These aren't actual PyUnit unit tests, but it's a similar idea.

In [2]:
def strindex(text, val):
    if (text+val).index(val) == len(text):
        return -1
    return text.index(val)
text = "Where is the U.S. located? I have no idea."
[y.lower() for y in nltk.word_tokenize(text) if strindex(string.punctuation,y) == -1 ] 

['where', 'is', 'the', 'u.s.', 'located', 'i', 'have', 'no', 'idea']

In [3]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [3]:
class MySearchEngine():
    def __init__(self):
        # Dict[str, str]: maps document id to original/raw text
        self.raw_text = {}
        
        # Dict[str, Counter]: maps document id to term vector (counts of terms in document)
        self.term_vectors = {}
        
        # Counter: maps term to count of how many documents contain term
        self.doc_freq = Counter()
        
        # Dict[str, set]: maps term to set of ids of documents that contain term
        self.inverted_index = defaultdict(set)
    
    # ------------------------------------------------------------------------
    #  indexing
    # ------------------------------------------------------------------------
    def strindex(self, text, val):
        if (text+val).index(val) == len(text):
            return -1
        return text.index(val)
    def tokenize(self, text):
        """ Converts text into tokens (also called "terms" or "words").
        
            This function should also handle normalization, e.g., lowercasing and 
            removing punctuation.
        
            For example, "The cat in the hat." --> ["the", "cat", "in", "the", "hat"]
        
            Parameters
            ----------
            text: str
                The string to separate into tokens.
        
            Returns
            -------
            list(str)
                A list where each element is a token.
        
        """
        # Hint: use NLTK's recommended word_tokenize() then filter out punctuation
        # It uses Punkt for sentence splitting and then tokenizes each sentence.
        # You'll notice that it's able to differentiate between an end-of-sentence period 
        # versus a period that's part of an abbreviation (like "U.S.").
        
        # tokenize
        ### YOUR CODE HERE ###
        
        # lowercase and filter out punctuation (using string.punctuation)
        
        
        
        ### YOUR CODE HERE ###
        
        return [y.lower() for y in nltk.word_tokenize(text) if self.strindex(string.punctuation,y) == -1] 


    def add(self, id, text):
        """ Adds document to index.
        
            Parameters
            ----------
            id: str
                A unique identifier for the document to add, e.g., the URL of a webpage.
            text: str
                The text of the document to be indexed.
        """
        
        if id in self.raw_text:
            raise LookupError

        self.raw_text[id] = text
        
        tokens = self.tokenize(text)
        
        c = Counter(tokens)
        
        self.term_vectors[id] = c
        
        for vv in c:
            self.inverted_index[vv].update([id])
        

        storage = set()
        storage.update( c.elements() )
        for varK in storage:
            self.doc_freq[varK] += 1
            
        
        
    def remove(self, id):
        """ Removes document from index.
        
            Parameters
            ----------
            id: str
                The identifier of the document to remove from the index.
        """
        # check if document exists and throw exception if so
        
        ### YOUR CODE HERE ###

        # remove raw text for this document
        del self.raw_text[id]
        ### YOUR CODE HERE ###

        #self.raw_text[id] = text
        

        c = self.term_vectors[id]
        
        for vv in self.term_vectors[id]:
            self.inverted_index[vv] -= set([id])
        

        storage = set()
        storage.update( c.elements() )
        for varK in storage:
            self.doc_freq[varK] -= 1
            
        # update document frequencies for terms found in this doc
        # i.e., counts should decrease by 1 for each (unique) term in term vector

        ### YOUR CODE HERE ###

        # remove term vector for this doc
        del self.term_vectors[id]
        ### YOUR CODE HERE ###

    def get(self, id):
        """ Returns the original (raw) text of a document.
        
            Parameters
            ----------
            id: str
                The identifier of the document to return.
        """
        
        return self.raw_text[id]
        # check if document exists and throw exception if so

        ### YOUR CODE HERE ###

        # return raw text

        ### YOUR CODE HERE ###
    
    def num_docs(self):
        """ Returns the current number of documents in index. 
        """
        return len(self.raw_text)
        ### YOUR CODE HERE ###

    # ------------------------------------------------------------------------
    #  matching
    # ------------------------------------------------------------------------

    def get_matches_term(self, term):
        """ Returns ids of documents that contain term.
        
            Parameters
            ----------
            term: str
                A single token, e.g., "cat" to match on.
            
            Returns
            -------
            set(str)
                A set of ids of documents that contain term.
        """
        term = term.lower()
        
        return self.inverted_index[term]
        
        # NOTE: term needs to be lowercased so can match output of tokenizer
        # look up term in inverted index

        ### YOUR CODE HERE ###

    def get_matches_OR(self, terms):
        """ Returns set of documents that contain at least one of the specified terms.
        
            Parameters
            ----------
            terms: iterable(str)
                An iterable of terms to match on, e.g., ["cat", "hat"].
            
            Returns
            -------
            set(str)
                A set of ids of documents that contain at least one of the term.
        """
        s = set()
        for term in terms:
            t = term.lower()
            s.update(self.inverted_index[t])
        return s
        # initialize set of ids to empty set

        ### YOUR CODE HERE ###
        
        # union ids with sets of ids matching any of the terms

        ### YOUR CODE HERE ###
    
    def get_matches_AND(self, terms):
        """ Returns set of documents that contain all of the specified terms.
        
            Parameters
            ----------
            terms: iterable(str)
                An iterable of terms to match on, e.g., ["cat", "hat"].
            
            Returns
            -------
            set(str)
                A set of ids of documents that contain each term.
        """ 
        s = set()
        s.update(self.inverted_index[terms[0]])
        for term in terms:
            t = term.lower()
            s &= (self.inverted_index[t])
        return s
        # initialize set of ids to those that match first term

        ### YOUR CODE HERE ###
        
        # intersect with sets of ids matching rest of terms

        ### YOUR CODE HERE ###
    
    def get_matches_NOT(self, terms):
        """ Returns set of documents that don't contain any of the specified terms.
        
            Parameters
            ----------
            terms: iterable(str)
                An iterable of terms to avoid, e.g., ["cat", "hat"].
            
            Returns
            -------
            set(str)
                A set of ids of documents that don't contain any of the terms.
        """
        ids = set(self.raw_text.keys())
        #print(ids)
        for term in terms:
            #print(set(self.inverted_index[term.lower()]))
            ids -= set(self.inverted_index[term.lower()])
        return ids

    # ------------------------------------------------------------------------
    #  scoring
    # ------------------------------------------------------------------------
        
    def idf(self, term):
        """ Returns current inverse document frequency weight for a specified term.
        
            Parameters
            ----------
            term: str
                A term.
            
            Returns
            -------
            float
                The value idf(t, D) as defined above.
        """ 
        N = len(self.raw_text.keys())
        n = self.doc_freq[term]
        
        return np.log10(N/(1+n))
        ### YOUR CODE HERE ###
    
    def dot_product(self, tv1, tv2):
        """ Returns dot product between two term vectors (including idf weighting).
        
            Parameters
            ----------
            tv1: Counter
                A Counter that contains term frequencies for terms in document 1.
            tv2: Counter
                A Counter that contains term frequencies for terms in document 2.
            
            Returns
            -------
            float
                The dot product of documents 1 and 2 as defined above.
        """
        total = 0
        for t in tv1:
            total += self.idf(t) * tv1[t]*tv2[t] * self.idf(t)
        
        return total
        # iterate over terms of one document
        # if term is also in other document, 
        # then add their product (tfidf(t,d1) * tfidf(t,d2)) to a running total

        ### YOUR CODE HERE ###
    
    def length(self, tv):
        """ Returns the length of a document (including idf weighting).
        
            Parameters
            ----------
            tv: Counter
                A Counter that contains term frequencies for terms in the document.
            
            Returns
            -------
            float
                The length of the document as defined above.
        """
        total = 0
        for t in tv:
            total += (tv[t]*self.idf(t))**2
            
        return total ** 0.5
        ### YOUR CODE HERE ###
    
    def cosine_similarity(self, tv1, tv2):
        """ Returns the cosine similarity (including idf weighting).

            Parameters
            ----------
            tv1: Counter
                A Counter that contains term frequencies for terms in document 1.
            tv2: Counter
                A Counter that contains term frequencies for terms in document 2.
            
            Returns
            -------
            float
                The cosine similarity of documents 1 and 2 as defined above.
        """
        dot = self.dot_product(tv1,tv2)
        mags = self.length(tv1) * self.length(tv2)
        
        return dot/mags
        
        ### YOUR CODE HERE ###

    # ------------------------------------------------------------------------
    #  querying
    # ------------------------------------------------------------------------

    def query(self, q, k=10):
        """ Returns up to top k documents matching at least one term in query q, sorted by relevance.
        
            Parameters
            ----------
            q: str
                A string containing words to match on, e.g., "cat hat".
        
            Returns
            -------
            List(tuple(str, float))
                A list of (document, score) pairs sorted in descending order.
                
        """
        # tokenize query
        tokens = self.tokenize(q)

        # get matches (just support OR style queries for now...)
        matches = self.get_matches_OR(tokens)
                
        # convert query to a term vector (Counter over tokens)
        c = Counter(tokens)
        
        scores = []
        # score each match by computing cosine similarity between query and document
        for m in matches:
            scores.append( ( m, self.cosine_similarity(c,self.term_vectors[m]) ) )
        ### YOUR CODE HERE ###
        scores.sort(key=itemgetter(1))
        scores = scores[::-1]
        # sort results and return top k
        return scores[:k]
        ### YOUR CODE HERE ###
    def advanced_query(self, q, k=10,require=None,exclude=None):
        """ Returns up to top k documents matching at least one term in query q, sorted by relevance.
        
            Parameters
            ----------
            q: str
                A string containing words to match on, e.g., "cat hat".
                +word means require word
                -word means require not having word
                
            
        
            Returns
            -------
            List(tuple(str, float))
                A list of (document, score) pairs sorted in descending order.
                
        """
        doAnd = False
        doNot = False
        if require != None:
            ands = self.get_matches_AND( self.tokenize(require) )
            doAnd = True
        if exclude != None:
            nots = self.get_matches_NOT( self.tokenize(exclude) )
            doNot = True
        # tokenize query
        tokens = self.tokenize(q)

        # get matches (just support OR style queries for now...)
        matches = self.get_matches_OR(tokens)
                
        if doAnd:
            fix_matches = (matches & ands) 
        if doNot:
            fix_matches = (matches & nots)
        # convert query to a term vector (Counter over tokens)
        c = Counter(tokens)
        
        scores = []
        # score each match by computing cosine similarity between query and document
        for m in fix_matches:
            scores.append( ( m, self.cosine_similarity(c,self.term_vectors[m]) ) )
        ### YOUR CODE HERE ###
        scores.sort(key=itemgetter(1))
        scores = scores[::-1]
        # sort results and return top k
        return scores[:k]
        ### YOUR CODE HERE ###



## 2 Adding documents

This section contains examples of adding documents and retrieving their raw text by id.

2.0 Initialize search engine and add sample documents.

In [4]:
engine = MySearchEngine()
engine.add("d1", "The cat in the hat.")
engine.add("d2", "Which cat wrote the cat book?")
engine.add("d3", "A dog is wearing a hat and reading a book!")
engine.add("d4", "Her day started off with a good book.")
engine.add("d5", "My days are numbered.")

NameError: name 'Counter' is not defined

2.1 Verify that document count is 5.

In [5]:
engine.num_docs()
# SOLUTION

5

2.2 Verify that retrieving document "d1" (by id) returns the string: "The cat in the hat."

In [6]:
# SOLUTION
engine.get("d6")

KeyError: 'd6'

2.3 Verify that adding a document with an id that's already been used throws an exception.

In [7]:
# SOLUTION
engine.add("d1", "The cat in the hat.")

LookupError: 

2.4 Verify that removing a document by id reduces the number of documents by 1.

In [8]:
# SOLUTION
engine.remove('d1')
engine.num_docs()

4

2.5 Verify that trying to retrieve that document again (by id) throws an exception.

In [9]:
# SOLUTION
engine.get('d1')

KeyError: 'd1'

2.7 Verify that we can add document d5 ("My days are numbered.") back in.

In [10]:
# SOLUTION
engine.add("d1", "The cat in the hat.")

2.7 Verify tokenization by tokenizing the string "The U.S. was represented at the meeting." This should yield:

```python
['the', 'u.s.', 'was', 'represented', 'at', 'the', 'meeting']
```

In [11]:
# SOLUTION
engine.tokenize("The U.S. was represented at the meeting.")

['the', 'u.s.', 'was', 'represented', 'at', 'the', 'meeting']

## 3 Matching documents

3.1 Verify that matching on single term "cat" returns `{'d1', 'd2'}`.

In [12]:
# SOLUTION
engine.get_matches_term("cat")

{'d1', 'd2'}

3.2 Verify that matching on "cat" or "dog" returns `{'d1', 'd2', 'd3'}`.

In [13]:
engine.get_matches_OR(["cat","dog"])

{'d1', 'd2', 'd3'}

3.3 Verify that matching on "cat" and "hat" returns `{'d1'}`.

In [14]:
engine.get_matches_AND(["cat","hat"])

{'d1'}

3.4 Verify that matching documents that do NOT contain "cat" or "dog" returns `{'d4', 'd5'}`.

In [15]:
# SOLUTION
engine.get_matches_NOT(["cat","hat"])

{'d4', 'd5'}

## 4 Query result scoring and ranking

4.1 Verify that the idf of "cat" is `0.221848749616` and the idf of "book" is `0.0969100130081` (assuming d1, ..., d5 from above). Does it make sense that the weight for "cat" is higher?

In [16]:
# SOLUTION
engine.idf("cat")

0.22184874961635639

4.2 Verify that a query for "cat" returns: 
```python
[('d2', 0.58656677583588812), 
 ('d1', 0.32937676397011012)]
```
Why is d2 ranked higher than d1?

In [17]:
# SOLUTION
print(engine.query("cat"))

[('d2', 0.58656677583588812), ('d1', 0.32937676397011012)]


4.3 Verify that a query for "cat book" returns:
```python
[('d2', 0.58880446883493398),
 ('d1', 0.30183523984479854),
 ('d4', 0.038624814550947489),
 ('d3', 0.034111490780396964)]
```
Does this ordering make sense? Think about how idf and also the length of documents affect scoring.

In [18]:
# SOLUTION
print(engine.query("cat book"))

[('d2', 0.58880446883493398), ('d1', 0.30183523984479854), ('d4', 0.038624814550947489), ('d3', 0.034111490780396964)]


In [25]:
print(engine.advanced_query("dog cat book",require="hat"))

[('d3', 0.31666912632857153), ('d1', 0.15687561261972546)]


In [1]:
print(engine.advanced_query("dog cat",exclude="book"))

NameError: name 'engine' is not defined

4.4 Possible ideas for extra time:

- implement a query language that allows `"+"` to mean "must have" and `"-"` to mean "must not have"; e.g., `"cat -book"` would return all documents that contain cat but not book
- incorporate stemming (or lemmatization) into tokenization (see NLTK documentation)
- incorporate stopword removal
- think about how phrase search could be supported (i.e., require that terms in a quoted phrase  appear next to each other in matching documents, e.g., like "burger king")
- ingest a set of documents (e.g., song lyrics) and see if ranked results look good for various queries